In [249]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [250]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from collections import deque
from oracle import make_oracle
from step_search import wolfe_line_search
from optimize import optimize_newton

In [251]:
# BFGS!
def bfgs_update(H, I, s, y):
    p = 1 / (y.T @ s)
    V = I - p * np.outer(y, s)
    
    H_new = V.T @ H @ V + p * np.outer(s, s)
    
    return H_new

def bfgs(oracle, w, tol=1e-8, gamma=30, max_iter=10000, verbose=False):
    I = np.identity(oracle.dim)
    H = gamma * I
    
    prev_w, prev_grad = w, oracle.grad(w).reshape(-1, 1)
    for iter_ in range(max_iter):
        if prev_grad.T @ prev_grad <= tol:
            break
        
        direction = H @ prev_grad
        alpha = wolfe_line_search(oracle, w, direction)
        
        if iter_ == 0:
            alpha = 1.0

        w = w - alpha * direction
        
        if verbose:
            print(f"Iteration {iter_}: {oracle.value(w)}, alpha: {alpha}")
        
        new_grad = oracle.grad(w).reshape(-1, 1)
                
        s = w - prev_w
        y = new_grad - prev_grad
        
        if iter_ == 0:
            H = ((y.T @ s) / (y.T @ y)) * H
        
        H = bfgs_update(H, I, s, y)        
        prev_w, prev_grad = w, new_grad

        
    return w

In [252]:
oracle = make_oracle("data/a1a.txt")
w_init = np.zeros((oracle.dim, 1))

In [253]:
%%time
bfgs(oracle, w_init, tol=1e-8, verbose=True);

Iteration 0: 6.799790183977318, alpha: 1.0
Iteration 1: 3.155299174894922, alpha: [[0.50000655]]
Iteration 2: 2.3505144133758664, alpha: 1.0
Iteration 3: 2.071314449921163, alpha: [[0.49962633]]
Iteration 4: 1.966200565086752, alpha: 1.0
Iteration 5: 1.4379936797961366, alpha: 1.0
Iteration 6: 1.0651342962372563, alpha: 1.0
Iteration 7: 0.9978287153805063, alpha: 1.0
Iteration 8: 0.7711179108123621, alpha: [[0.49970057]]
Iteration 9: 0.6906229594605998, alpha: [[0.49924556]]
Iteration 10: 0.6132023475534327, alpha: [[0.49952347]]
Iteration 11: 0.5695093935307574, alpha: 1.0
Iteration 12: 0.49650466649296454, alpha: [[0.49851075]]
Iteration 13: 0.4807835243528352, alpha: [[0.49900374]]
Iteration 14: 0.45029390905580396, alpha: [[0.21079889]]
Iteration 15: 0.4365371493742264, alpha: [[0.49902375]]
Iteration 16: 0.4122754986840929, alpha: [[0.21051086]]
Iteration 17: 0.3975260759125425, alpha: [[0.21075402]]
Iteration 18: 0.38838415284012656, alpha: [[0.2110458]]
Iteration 19: 0.385841540

array([[-1.51538753e+00],
       [-7.64074136e-01],
       [-2.35044698e-01],
       [ 4.46205571e-01],
       [ 1.99601032e-01],
       [-1.05342165e-01],
       [-4.96974055e-01],
       [-7.16873072e-02],
       [ 6.49719345e-01],
       [ 1.84028670e-01],
       [-6.99898393e-01],
       [ 0.00000000e+00],
       [-1.97934280e-01],
       [-6.02854672e-01],
       [-2.68250184e-01],
       [-3.51926688e-01],
       [-8.58954258e-02],
       [-5.59772786e-01],
       [-7.67354344e-01],
       [ 2.55262676e-01],
       [ 4.68483815e+00],
       [ 2.22771359e-01],
       [ 1.12169257e+00],
       [-3.72158734e-01],
       [ 8.33748671e-01],
       [-6.13120372e+00],
       [ 4.67376495e+00],
       [-2.60052548e+00],
       [-2.26330765e-01],
       [-1.33002890e+00],
       [ 3.84621984e+00],
       [ 2.06321932e+00],
       [-6.90031931e+00],
       [-1.24229606e+00],
       [-4.99955052e+00],
       [ 2.22771359e-01],
       [ 2.55262676e-01],
       [ 4.61589937e-01],
       [ 2.1

In [259]:
def lbfgs_direction(grad, H0, s_q, y_q, r_q):
    alpha = np.zeros(len(s_q))
    
    q = grad
    for i in range(len(s_q) - 1, -1, -1):
        alpha[i] = r_q[i] * s_q[i].T @ q
        q = q - alpha[i] * y_q[i]
        
    Hd = H0 @ q
    for i in range(len(s_q)):
        beta = r_q[i] * y_q[i].T @ Hd
        Hd = Hd + s_q[i] @ (alpha[i] - beta)    
    
    return Hd

def lbfgs(oracle, w, tol=1e-8, buffer_size=5, gamma=1.0, max_iter=10000, verbose=False):
    s_q, y_q, r_q = [deque(maxlen=buffer_size) for _ in range(3)]
    I = np.identity(oracle.dim)
    
    prev_w, prev_grad = w, oracle.grad(w).reshape(-1, 1)
    for iter_ in range(max_iter):
        if prev_grad.T @ prev_grad <= tol:
            break
        
        if iter_ == 0:
            H0 = gamma * I
        else:
            H0 = (y_q[-1].T @ s_q[-1]) / (y_q[-1].T @ y_q[-1]) * I
    
        direction = lbfgs_direction(prev_grad, H0, s_q, y_q, r_q)
        alpha = wolfe_line_search(oracle, w, direction)
        
        w = w - alpha * direction
        
        if verbose:
            print(f"Iteration {iter_}: {oracle.value(w)}, alpha: {alpha}")
             
        new_grad = oracle.grad(w).reshape(-1, 1)
        
        s_q.append(w - prev_w)
        y_q.append(new_grad - prev_grad)
        r_q.append(1 / (y_q[-1].T @ s_q[-1]))
        
        prev_w, prev_grad = w, new_grad
    
    return w

In [274]:
%%time
_ = lbfgs(oracle, w_init, buffer_size=100, gamma=1.0)

CPU times: user 620 ms, sys: 54.1 ms, total: 674 ms
Wall time: 355 ms
